<a href="https://colab.research.google.com/github/irdinainsharuki/MLBCnittanyAI/blob/main/FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading data

In [4]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np

In [5]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-11-01 18:49:30--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.207, 173.194.217.207, 108.177.11.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   169MB/s    in 0.4s    

2024-11-01 18:49:31 (169 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Reading the Data into arrays

In [7]:
from PIL import Image
import numpy as np


cats_dir = os.path.join(train_dir + "/cats")
dogs_dir = os.path.join(train_dir + "/dogs")

x_train = []
y_train = []
NUMBER_OF_EXAMPLES =1000

i = 0
while i < NUMBER_OF_EXAMPLES:
  if i % 2 == 0:
    im = Image.open(os.path.join(cats_dir, os.listdir(cats_dir)[i])).convert("RGB")
    im_resized = im.resize((128, 128))

    x_train.append(np.array(im_resized))
    y_train.append(1)
  else:
    im = Image.open(os.path.join(dogs_dir, os.listdir(dogs_dir)[i])).convert("RGB")

    x_train.append(np.array(im_resized))
    y_train.append(0)
  i += 1

x_train = np.array(x_train).reshape(-1, 128, 128, 3)
y_train = np.array(y_train)



## Beginning to define the model (this is where you come in, I loaded the pretrained model for you)

In [ ]:
import tensorflow as tf

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(128, 128, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)

# Adding custom layers on top of the pretrained model for binary classification
x = pretrained_model.output
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=pretrained_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model to check the architecture
model.summary()



In [9]:
# Data Augmentation and normalization for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow(
    x_train, y_train,
    batch_size=16,
    subset='training'
)

validation_generator = train_datagen.flow(
    x_train, y_train,
    batch_size=16,
    subset='validation'
)

# Training the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

predictions = model.predict(x_train[:10])
predicted_classes = [1 if prob > 0.5 else 0 for prob in predictions.flatten()]
print('Sample predictions:', predicted_classes)

# Evaluate the model's accuracy on training data
loss, accuracy = model.evaluate(x_train, y_train)
print(f'Training accuracy: {accuracy:.2f}')

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


50/50 ━━━━━━━━━━━━━━━━━━━━ 281s 5s/step - accuracy: 0.4907 - loss: 1.2148 - val_accuracy: 0.5000 - val_loss: 0.7290
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 265s 5s/step - accuracy: 0.4556 - loss: 0.7020 - val_accuracy: 0.5000 - val_loss: 0.7475
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 253s 5s/step - accuracy: 0.4617 - loss: 0.6968 - val_accuracy: 0.5000 - val_loss: 0.6965
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 262s 5s/step - accuracy: 0.4818 - loss: 0.6944 - val_accuracy: 0.5000 - val_loss: 0.8633
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 260s 5s/step - accuracy: 0.4951 - loss: 0.6941 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 262s 5s/step - accuracy: 0.4860 - loss: 0.6941 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 227s 5s/step - accuracy: 0.4544 - loss: 0.6937 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 227s 5s/step - accuracy: 0.4811 - loss: 0.6937 - val_accuracy: 0.5000 - val_loss: 0.